In [1]:
# import das bibliotecas
import pyspark
from pyspark.sql.types import *
from pyspark.sql.functions import *
from pyspark import SparkConf, SparkContext
from pyspark.sql import SparkSession
import collections
import os
from os.path import isfile, isdir, join

In [11]:
# inicialização do spark
conf = SparkConf() \
        .setMaster("local[2]") \
        .setAppName("LendoDB") \
        .set("spark.executor.memory", "4g") \
        .set("spark.driver.memory", "4g") \
        .set("spark.ui.enabled", "true") \
        .set("spark.sql.shuffle.partitions" , "800") \

spark = SparkSession \
    .builder \
    .config(conf=conf) \
    .getOrCreate()

sc = spark.sparkContext

ERROR:root:Exception while sending command.
Traceback (most recent call last):
  File "/usr/local/spark/python/lib/py4j-0.10.9-src.zip/py4j/java_gateway.py", line 1207, in send_command
    raise Py4JNetworkError("Answer from Java side is empty")
py4j.protocol.Py4JNetworkError: Answer from Java side is empty

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/spark/python/lib/py4j-0.10.9-src.zip/py4j/java_gateway.py", line 1033, in send_command
    response = connection.send_command(command)
  File "/usr/local/spark/python/lib/py4j-0.10.9-src.zip/py4j/java_gateway.py", line 1211, in send_command
    raise Py4JNetworkError(
py4j.protocol.Py4JNetworkError: Error while receiving


Py4JError: SparkConf does not exist in the JVM

In [3]:
# definindo o schema dos dados
schema = StructType([
    StructField("dadosBasicos", StructType([
        StructField("assunto", ArrayType(
            StructType([
                StructField("assuntoLocal", StructType([
                    StructField("codigoAssunto", LongType(), True),
                    StructField("codigoPaiNacional", LongType(), True),
                    StructField("descricao", StringType(), True)
                ]), True),
                StructField("codigoNacional", LongType(), True),
                StructField("principal", BooleanType(), True)
            ]),
        ), True),
        StructField('classeProcessual', LongType(), True),
        StructField('codigoLocalidade', StringType(), True),
        StructField('competencia', StringType(), True),
        StructField('dataAjuizamento', StringType(), True),
        StructField('dscSistema', StringType(), True),
        StructField('nivelSigilo', LongType(), True),
        StructField('numero', StringType(), True),
        StructField("orgaoJulgador", StructType([
            StructField("codigoMunicipioIBGE", LongType(), True),
            StructField("codigoOrgao", StringType(), True),
            StructField("instancia", StringType(), True),
            StructField("nomeOrgao", StringType(), True)
        ]), True),
        StructField('procEl', LongType(), True),
        StructField("tamanhoProcesso", StringType(), True),
        StructField("totalAssuntos", LongType(), True),
        StructField("valorCausa", StringType(), True)       
    ]), True),
    StructField("grau", StringType(), True),
    StructField("millisInsercao", LongType(), True),
    StructField("movimento", ArrayType(     
        StructType([
            StructField("complementoNacional", ArrayType(
                StructType([
                    StructField("codComplemento", LongType(), True),
                    StructField("codComplementoTabelado", LongType(), True),
                    StructField("descricaoComplemento", StringType(), True),
                ])
            ), True),
            StructField("dataHora", StringType(), True),
            StructField("idDocumentoVinculado", ArrayType(
                StringType(),
            ), True),
            StructField("identificadorMovimento", StringType(), True),
            StructField("movimentoLocal", StructType([
                StructField('codigoMovimento', LongType(), True),
                StructField('codigoPaiNacional', LongType(), True)
            ]), True),
            StructField("movimentoNacional", StructType([
                StructField('codigoNacional', LongType(), True)
            ]), True),
            StructField("nivelSigilo", StringType(), True),
            StructField("orgaoJulgador", StringType(), True),
            StructField("tipoDecisao", StringType(), True),
            StructField("tipoResponsavelMovimento", StringType(), True)
        ]),
    ), True),
    StructField("siglaTribunal", StringType(), True)
])

In [121]:
# testando o carregamento de um arquivo com o schema
#df_with_schema = spark.read.json("./base/justica_federal/processos-trf1/processos-trf1_1.json")
#df_with_schema = spark.read.json("./base/justica_federal/processos-trf1/processos-trf1_1.json")

df_with_schema = spark.read.schema(schema).json("./base/justica_estadual/processos-tjpb/processos-tjpb_2.json")
#df_with_schema = spark.read.json("./base/justica_estadual/processos-tjpb/processos-tjpb_2.json")
df_with_schema.printSchema()
df_with_schema.take(1)

root
 |-- dadosBasicos: struct (nullable = true)
 |    |-- assunto: array (nullable = true)
 |    |    |-- element: struct (containsNull = true)
 |    |    |    |-- assuntoLocal: struct (nullable = true)
 |    |    |    |    |-- codigoAssunto: long (nullable = true)
 |    |    |    |    |-- codigoPaiNacional: long (nullable = true)
 |    |    |    |    |-- descricao: string (nullable = true)
 |    |    |    |-- codigoNacional: long (nullable = true)
 |    |    |    |-- principal: boolean (nullable = true)
 |    |-- classeProcessual: long (nullable = true)
 |    |-- codigoLocalidade: string (nullable = true)
 |    |-- competencia: string (nullable = true)
 |    |-- dataAjuizamento: string (nullable = true)
 |    |-- dscSistema: string (nullable = true)
 |    |-- nivelSigilo: long (nullable = true)
 |    |-- numero: string (nullable = true)
 |    |-- orgaoJulgador: struct (nullable = true)
 |    |    |-- codigoMunicipioIBGE: long (nullable = true)
 |    |    |-- codigoOrgao: string (null

[Row(dadosBasicos=Row(assunto=[Row(assuntoLocal=None, codigoNacional=4960, principal=False)], classeProcessual=12154, codigoLocalidade='250430', competencia=None, dataAjuizamento='52110000000', dscSistema=None, nivelSigilo=0, numero='00000017519718150141', orgaoJulgador=Row(codigoMunicipioIBGE=0, codigoOrgao='6070', instancia='ORIG', nomeOrgao='1A. VARA DE CATOLE DO ROCHA'), procEl=None, tamanhoProcesso=None, totalAssuntos=None, valorCausa=None), grau='G1', millisInsercao=1560201450546, movimento=[Row(complementoNacional=None, dataHora='20190520000000', idDocumentoVinculado=None, identificadorMovimento='63', movimentoLocal=None, movimentoNacional=Row(codigoNacional=848), nivelSigilo=None, orgaoJulgador=None, tipoDecisao=None, tipoResponsavelMovimento=None), Row(complementoNacional=None, dataHora='20190520000000', idDocumentoVinculado=None, identificadorMovimento='64', movimentoLocal=None, movimentoNacional=Row(codigoNacional=22), nivelSigilo=None, orgaoJulgador=None, tipoDecisao=None, 

In [7]:
# carregamento de todos os arquivos em um único DataFrame
basedir = "./base"

dirs_ramos_justica = [join(basedir, f) for f in os.listdir(basedir) if isdir(join(basedir, f))]

lista_arquivos = list()

dfUnion = spark.createDataFrame(spark.sparkContext.emptyRDD(),schema)

for dir_ramo_just in dirs_ramos_justica:
    dirs_tribunais = [join(dir_ramo_just, f) for f in os.listdir(dir_ramo_just) if isdir(join(dir_ramo_just, f))]
    
    for dir_trib in dirs_tribunais:
        arquivos = [join(dir_trib, f) for f in os.listdir(dir_trib) if isfile(join(dir_trib, f))]
        lista_arquivos.extend(arquivos)
        
        for arq in arquivos:
            print("Carregando dataFrame do arquivo: " + arq)
            df = spark.read.schema(schema).json(arq)
            dfUnion = dfUnion.union(df)

Carregando dataFrame do arquivo: ./base/justica_estadual/processos-tjba/processos-tjba_3.json
Carregando dataFrame do arquivo: ./base/justica_estadual/processos-tjba/processos-tjba_10.json
Carregando dataFrame do arquivo: ./base/justica_estadual/processos-tjba/processos-tjba_11.json
Carregando dataFrame do arquivo: ./base/justica_estadual/processos-tjba/processos-tjba_2.json
Carregando dataFrame do arquivo: ./base/justica_estadual/processos-tjba/processos-tjba_9.json
Carregando dataFrame do arquivo: ./base/justica_estadual/processos-tjba/processos-tjba_5.json
Carregando dataFrame do arquivo: ./base/justica_estadual/processos-tjba/processos-tjba_4.json
Carregando dataFrame do arquivo: ./base/justica_estadual/processos-tjba/processos-tjba_8.json
Carregando dataFrame do arquivo: ./base/justica_estadual/processos-tjba/processos-tjba_7.json
Carregando dataFrame do arquivo: ./base/justica_estadual/processos-tjba/processos-tjba_6.json
Carregando dataFrame do arquivo: ./base/justica_estadual/p

Carregando dataFrame do arquivo: ./base/justica_estadual/processos-tjes/processos-tjes_3.json
Carregando dataFrame do arquivo: ./base/justica_estadual/processos-tjes/processos-tjes_11.json
Carregando dataFrame do arquivo: ./base/justica_estadual/processos-tjes/processos-tjes_10.json
Carregando dataFrame do arquivo: ./base/justica_estadual/processos-tjes/processos-tjes_2.json
Carregando dataFrame do arquivo: ./base/justica_estadual/processos-tjap/processos-tjap_6.json
Carregando dataFrame do arquivo: ./base/justica_estadual/processos-tjap/processos-tjap_7.json
Carregando dataFrame do arquivo: ./base/justica_estadual/processos-tjap/processos-tjap_11.json
Carregando dataFrame do arquivo: ./base/justica_estadual/processos-tjap/processos-tjap_1.json
Carregando dataFrame do arquivo: ./base/justica_estadual/processos-tjap/processos-tjap_10.json
Carregando dataFrame do arquivo: ./base/justica_estadual/processos-tjap/processos-tjap_2.json
Carregando dataFrame do arquivo: ./base/justica_estadual

Carregando dataFrame do arquivo: ./base/justica_estadual/processos-tjam/processos-tjam_8.json
Carregando dataFrame do arquivo: ./base/justica_estadual/processos-tjam/processos-tjam_4.json
Carregando dataFrame do arquivo: ./base/justica_estadual/processos-tjam/processos-tjam_10.json
Carregando dataFrame do arquivo: ./base/justica_estadual/processos-tjam/processos-tjam_3.json
Carregando dataFrame do arquivo: ./base/justica_estadual/processos-tjam/processos-tjam_2.json
Carregando dataFrame do arquivo: ./base/justica_estadual/processos-tjam/processos-tjam_11.json
Carregando dataFrame do arquivo: ./base/justica_estadual/processos-tjam/processos-tjam_12.json
Carregando dataFrame do arquivo: ./base/justica_estadual/processos-tjam/processos-tjam_1.json
Carregando dataFrame do arquivo: ./base/justica_estadual/processos-tjam/processos-tjam_7.json
Carregando dataFrame do arquivo: ./base/justica_estadual/processos-tjam/processos-tjam_6.json
Carregando dataFrame do arquivo: ./base/justica_estadual/

Carregando dataFrame do arquivo: ./base/justica_estadual/processos-tjsp/processos-tjsp_9.json
Carregando dataFrame do arquivo: ./base/justica_estadual/processos-tjsp/processos-tjsp_6.json
Carregando dataFrame do arquivo: ./base/justica_estadual/processos-tjsp/processos-tjsp_7.json
Carregando dataFrame do arquivo: ./base/justica_estadual/processos-tjsp/processos-tjsp_12.json
Carregando dataFrame do arquivo: ./base/justica_estadual/processos-tjsp/processos-tjsp_1.json
Carregando dataFrame do arquivo: ./base/justica_estadual/processos-tjrs/processos-tjrs_6.json
Carregando dataFrame do arquivo: ./base/justica_estadual/processos-tjrs/processos-tjrs_7.json
Carregando dataFrame do arquivo: ./base/justica_estadual/processos-tjrs/processos-tjrs_11.json
Carregando dataFrame do arquivo: ./base/justica_estadual/processos-tjrs/processos-tjrs_1.json
Carregando dataFrame do arquivo: ./base/justica_estadual/processos-tjrs/processos-tjrs_10.json
Carregando dataFrame do arquivo: ./base/justica_estadual/

In [8]:
# cria uma view
dfUnion.createOrReplaceTempView("processos")

In [177]:
#scala> val oneDF = df.select(col("data"), explode(col("data"))).toDF("data", "element").select(col("data"), col("element.parent"))
#scala> oneDF.show

testDF = dfUnion.select(
    col("dadosBasicos.numero"), 
    explode("dadosBasicos.assunto")
    ).toDF("NPU", "itemAssunto")
testDF.take(1)

[]

In [10]:
# SQL statements can be run by using the sql methods provided by spark
queryDF = spark.sql("SELECT * FROM processos WHERE siglaTribunal = 'TJPE'")
queryDF.take(5)

Py4JJavaError: An error occurred while calling o9760.collectToPython.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 47 in stage 5.0 failed 1 times, most recent failure: Lost task 47.0 in stage 5.0 (TID 173, 9d265b7bcee3, executor driver): java.lang.OutOfMemoryError: Java heap space
	at java.base/java.util.Arrays.copyOf(Arrays.java:3745)
	at org.apache.hadoop.io.Text.setCapacity(Text.java:266)
	at org.apache.hadoop.io.Text.append(Text.java:236)
	at org.apache.hadoop.util.LineReader.readDefaultLine(LineReader.java:245)
	at org.apache.hadoop.util.LineReader.readLine(LineReader.java:176)
	at org.apache.hadoop.mapreduce.lib.input.UncompressedSplitLineReader.readLine(UncompressedSplitLineReader.java:94)
	at org.apache.hadoop.mapreduce.lib.input.LineRecordReader.skipUtfByteOrderMark(LineRecordReader.java:152)
	at org.apache.hadoop.mapreduce.lib.input.LineRecordReader.nextKeyValue(LineRecordReader.java:192)
	at org.apache.spark.sql.execution.datasources.RecordReader

In [128]:
# SQL statements can be run by using the sql methods provided by spark
queryDF = spark.sql("SELECT * FROM processos WHERE dadosBasicos IS NULL")
queryDF.show()

AnalysisException: cannot resolve '(processos.`dadosBasicos`.`assunto`.`codigoNacional` = 4960)' due to data type mismatch: differing types in '(processos.`dadosBasicos`.`assunto`.`codigoNacional` = 4960)' (array<bigint> and int).; line 1 pos 30;
'Project [*]
+- 'Filter (dadosBasicos#2398.assunto.codigoNacional = 4960)
   +- SubqueryAlias processos
      +- Relation[dadosBasicos#2398,grau#2399,millisInsercao#2400L,movimento#2401,siglaTribunal#2402] json


AttributeError: 'list' object has no attribute 'foreach'